In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cxr-dataset/reports.csv
/kaggle/input/cxr-dataset/projections.csv
/kaggle/input/cxr-dataset/gpt-2-model_impressions/config.json
/kaggle/input/cxr-dataset/gpt-2-model_impressions/model.safetensors
/kaggle/input/cxr-dataset/gpt-2-model_impressions/generation_config.json
/kaggle/input/cxr-dataset/gpt-2-tokenizer_impressions/merges.txt
/kaggle/input/cxr-dataset/gpt-2-tokenizer_impressions/vocab.json
/kaggle/input/cxr-dataset/gpt-2-tokenizer_impressions/tokenizer_config.json
/kaggle/input/cxr-dataset/gpt-2-tokenizer_impressions/special_tokens_map.json
/kaggle/input/cxr-dataset/gpt-2-tokenizer_findings/merges.txt
/kaggle/input/cxr-dataset/gpt-2-tokenizer_findings/vocab.json
/kaggle/input/cxr-dataset/gpt-2-tokenizer_findings/tokenizer_config.json
/kaggle/input/cxr-dataset/gpt-2-tokenizer_findings/special_tokens_map.json
/kaggle/input/cxr-dataset/images/349_IM-1697-2001.dcm.png
/kaggle/input/cxr-dataset/images/607_IM-2196-1001.dcm.png
/kaggle/input/cxr-dataset/images/2832_IM-1249

In [5]:
import pandas as pd
import numpy as np
import os
import tqdm

from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import torchvision
from torchvision import datasets, models, transforms
from torchvision.io import read_image, ImageReadMode
from torchvision.datasets import ImageFolder

from transformers import GPT2Tokenizer
from transformers import GPT2Config
from transformers import GPT2LMHeadModel
from transformers import AdamW
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CPU


In [7]:
reports_df = pd.read_csv('/kaggle/input/cxr-dataset/reports.csv')
reports_df.head()

,uid,MeSH,Problems,image,indication,comparison,findings,impression
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
2,3,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
3,4,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
4,5,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.


In [8]:
projections_df = pd.read_csv('/kaggle/input/cxr-dataset/projections.csv')
projections_df.head()

,uid,filename,projection
0,1,1_IM-0001-4001.dcm.png,Frontal
1,1,1_IM-0001-3001.dcm.png,Lateral
2,2,2_IM-0652-1001.dcm.png,Frontal
3,2,2_IM-0652-2001.dcm.png,Lateral
4,3,3_IM-1384-1001.dcm.png,Frontal


In [9]:
uids = projections_df.uid.unique()
train_uids, test_uids = train_test_split(uids, test_size=0.1, random_state=42)

# Evaluation of Findings Language Model

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [11]:
test_findings_corpus = [tokenizer.encode(line)  for line in reports_df[reports_df.uid.isin(test_uids)].findings.dropna()]

In [12]:
pd.Series([len(x) for x in test_findings_corpus]).describe(percentiles=[0.25, 0.5, 0.75, 0.8, 0.9, 0.95])

count    334.000000
mean      51.583832
std       22.083322
min       15.000000
25%       37.000000
50%       48.000000
75%       62.750000
80%       66.000000
90%       81.000000
95%       91.350000
max      153.000000
dtype: float64

In [13]:
class GPT2Dataset(Dataset):
    def __init__(self, txt_list, max_length):
        self.input_ids = [torch.tensor(t[:max_length-1] + [tokenizer.eos_token_id]) for t in txt_list]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return self.input_ids[index]

def collate_batch(batch):
    """
    Pad the batch to ensure all sequences have the same length.
    """
    pad_id = tokenizer.pad_token_id
    if pad_id is None:
        pad_id = tokenizer.eos_token_id
    return pad_sequence(batch, batch_first=True, padding_value=pad_id)

# Create dataset
test_dataset = GPT2Dataset(test_findings_corpus, max_length=128)  # Adjust max_length as needed
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=collate_batch)

In [14]:
model = GPT2LMHeadModel.from_pretrained('/kaggle/input/cxr-dataset/gpt-2-model_tokenizer_findings')
tokenizer = GPT2Tokenizer.from_pretrained('/kaggle/input/cxr-dataset/gpt-2-tokenizer_findings')
model = model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [16]:
import math
total_loss = 0
total_length = 0
with torch.no_grad():
    for batch in test_dataloader:
        outputs = model(batch.to(device), labels=batch.to(device))
        loss = outputs.loss
        total_loss += loss.item() * batch.size(1)
        total_length += batch.size(1)

perplexity = math.exp(total_loss / total_length)
print(f'Perplexity: {perplexity}')


Perplexity: 1.9489982488623694


In [17]:
# Function to generate text from the model
def generate_text(prompt_text, max_length=100):
    encoded_input = tokenizer.encode(prompt_text, return_tensors='pt').to(device)
    generated_sequences = model.generate(
        input_ids=encoded_input,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(generated_sequences[0], skip_special_tokens=True)
    return generated_text

In [21]:
from nltk.translate.bleu_score import sentence_bleu
generated_texts = []
reference_texts = []
for test_sample in test_findings_corpus:
    # Join tokenized list into a string if necessary
    if isinstance(test_sample, list):
        test_sample = tokenizer.decode(test_sample, skip_special_tokens=True)
    prompt_text = " ".join(test_sample.split()[:5])
    generated_text = generate_text(prompt_text)
    generated_texts.append(generated_text)
    reference_texts.append(test_sample)

# Calculate BLEU scores
bleu_scores = [sentence_bleu([ref.split()], gen.split()) for ref, gen in zip(reference_texts, generated_texts)]
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f'Average BLEU Score: {average_bleu_score}')

Average BLEU Score: 0.3722908950014124


# Evaluation of Impressions Language Model

In [22]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [23]:
test_impressions_corpus = [tokenizer.encode(line)  for line in reports_df[reports_df.uid.isin(test_uids)].impression.dropna()]

In [24]:
pd.Series([len(x) for x in test_impressions_corpus]).describe(percentiles=[0.25, 0.5, 0.75, 0.8, 0.9, 0.95])

count    380.000000
mean      17.036842
std       16.291821
min        2.000000
25%        8.000000
50%       10.000000
75%       21.000000
80%       27.000000
90%       43.000000
95%       51.000000
max      118.000000
dtype: float64

In [25]:
class GPT2Dataset(Dataset):
    def __init__(self, txt_list, max_length):
        self.input_ids = [torch.tensor(t[:max_length-1] + [tokenizer.eos_token_id]) for t in txt_list]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return self.input_ids[index]

def collate_batch(batch):
    """
    Pad the batch to ensure all sequences have the same length.
    """
    pad_id = tokenizer.pad_token_id
    if pad_id is None:
        pad_id = tokenizer.eos_token_id
    return pad_sequence(batch, batch_first=True, padding_value=pad_id)

# Create dataset
test_dataset = GPT2Dataset(test_impressions_corpus, max_length=128)  # Adjust max_length as needed
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=collate_batch)

In [26]:
model = GPT2LMHeadModel.from_pretrained('/kaggle/input/cxr-dataset/gpt-2-model_impressions')
tokenizer = GPT2Tokenizer.from_pretrained('/kaggle/input/cxr-dataset/gpt-2-tokenizer_impressions')
model = model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [27]:
total_loss = 0
total_length = 0
with torch.no_grad():
    for batch in test_dataloader:
        outputs = model(batch.to(device), labels=batch.to(device))
        loss = outputs.loss
        total_loss += loss.item() * batch.size(1)
        total_length += batch.size(1)

perplexity = math.exp(total_loss / total_length)
print(f'Perplexity: {perplexity}')

Perplexity: 1.9978719323448744


In [28]:
# Function to generate text from the model
def generate_text(prompt_text, max_length=100):
    encoded_input = tokenizer.encode(prompt_text, return_tensors='pt').to(device)
    generated_sequences = model.generate(
        input_ids=encoded_input,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(generated_sequences[0], skip_special_tokens=True)
    return generated_text

In [29]:
generated_texts = []
reference_texts = []
for test_sample in test_impressions_corpus:
    # Join tokenized list into a string if necessary
    if isinstance(test_sample, list):
        test_sample = tokenizer.decode(test_sample, skip_special_tokens=True)
    prompt_text = " ".join(test_sample.split()[:5])
    generated_text = generate_text(prompt_text)
    generated_texts.append(generated_text)
    reference_texts.append(test_sample)

# Calculate BLEU scores
bleu_scores = [sentence_bleu([ref.split()], gen.split()) for ref, gen in zip(reference_texts, generated_texts)]
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f'Average BLEU Score: {average_bleu_score}')

Average BLEU Score: 0.7330372239916698


> # Findings Language model
#   BLEU Score of  0.3722908950014124
#   Perplexity of 1.9489982488623694

> # Impressions Language model
# BLEU Score of 0.7330372239916698
# Perplexity of 1.9978719323448744